## Bout mapping

In [1]:
from behavior_analysis.experiment import BehaviorExperiment
from behavior_analysis.analysis.bouts import BoutData
from behavior_analysis.analysis.bout_mapping import calculate_distance_matrix_templates, interpolate_nd
from behavior_analysis.utilities.timer import Timer
import pandas as pd
import numpy as np
from pathlib import Path
import os

In [ ]:
if __name__ == "__main__":
    # Open template bouts
    template_directory = Path(r'J:\Duncan Mearns\behavior_mapping')
    template_frame_rate = 500.
    # Open 1744 exemplar bouts representative of all behaviors
    exemplars_df = pd.read_csv(template_directory.joinpath('exemplars.csv'),
                               dtype={'ID': str, 'code': str})
    # Open tail statistics and eigenfish for bout mapping
    eigenfish = np.load(template_directory.joinpath('eigenfish.npy'))
    eigenfish = eigenfish[:3]  # take first three eigenfish only
    mean, std = np.load(template_directory.joinpath('tail_statistics.npy'))
    # Import template bouts
    templates = BoutData.from_metadata(exemplars_df, template_directory.joinpath("kinematics"))
    # Map template bouts onto eigenfish
    templates = templates.map(vectors=eigenfish, whiten=True, mean=mean, std=std)
    templates = templates.to_list(values=True)
    print(len(templates))
    print(templates[0].shape)

Opening 681 csv files...
############ 10%
############ 20%
############ 30%
############ 40%
############ 50%
############ 60%
###

In [ ]:
if __name__ == "__main__":
    # Open experiment
    experiment = BehaviorExperiment.open(r"C:\Users\manyung.ng\Documents\behaviour_analysis\behavior_analysis_tracking\test_analysis")
    print(experiment)
    # Open video and bout info
    video_info = pd.read_csv(experiment.directory.joinpath('video_data.csv'), dtype={'ID': str, 'code': str})
    bouts_df = pd.read_csv(experiment.subdirs["analysis"].joinpath('bouts.csv'),
                           dtype={'ID': str, 'code': str})

In [ ]:
if __name__ == "__main__":
    # Import bouts
    bouts = BoutData.from_metadata(bouts_df, experiment.subdirs["kinematics"], tail_only=True)
    # Map bouts onto eigenfish
    bouts = bouts.map(vectors=eigenfish, whiten=True, mean=mean, std=std)
    # Start timer
    timer = Timer()
    analysis_times = []
    timer.start()
    # Iterate through fish
    output_directory = experiment.subdirs["analysis"].joinpath('bout_distances')
    if not output_directory.exists():
        output_directory.mkdir(parents=True)
    for ID in bouts.metadata['ID'].unique():
        # Save distance matrix for each fish to bout_distances folder in analysis directory
        output_path = output_directory.joinpath(ID + '.npy')
        if not output_path.exists():
            print(ID + '...', end=' ')
            # Interpolate bouts to correct frame rate
            fish_bouts = []
            for i, bout in bouts.iter(IDs=[ID], values=True):
                code = bouts.metadata.loc[i, 'code']
                fps = video_info[video_info['code'] == code].squeeze().fps
                interp = interpolate_nd(bout, fps, template_frame_rate)
                fish_bouts.append(interp)
            # Calculate distance matrix
            D = calculate_distance_matrix_templates(fish_bouts, templates, fs=template_frame_rate)
            # Save distance matrix
            np.save(output_path, D)
            # Show time taken
            time_taken = timer.lap()
            analysis_times.append(time_taken)
            print(timer.convert_time(time_taken))
    average_time = timer.average
    print(f'Total time: {timer.convert_time(timer.stop())}')
    print(f'Average time: {timer.convert_time(average_time)}')

In [ ]:
# load the calculated distances
# only run if the previous chunk was completed and the session is restarted.
distances_dir = Path(r"C:\Users\manyung.ng\Documents\behaviour_analysis\behavior_analysis_tracking\test_analysis\analysis\bout_distances")
bouts = BoutData.from_metadata(bouts_df, experiment.subdirs["kinematics"], tail_only=True)
bouts = bouts.map(vectors=eigenfish, whiten=True, mean=mean, std=std)
for ID in bouts.metadata['ID'].unique():
    distances = np.load(distances_dir.joinpath(ID + '.npy'))
    print("Loaded.")

In [ ]:
# Assign exemplars
mapped_bouts = bouts.map(eigenfish, whiten=True, mean=mean, std=std)
mapped_bouts['exemplar'] = None
for ID, fish_distances in distances.iteritems():
    bout_idxs = mapped_bouts[mapped_bouts['ID'] == ID].index
    nearest_exemplar = np.argmin(fish_distances, axis=1)
    mapped_bouts.loc[bout_idxs, 'exemplar'] = nearest_exemplar
mapped_bouts.to_csv(os.path.join(experiment.subdirs['analysis'], 'mapped_bouts.csv'))

## Bout classification

In [ ]:
from behavior_analysis.experiment import BehaviorExperiment
from behavior_analysis.analysis.bouts import BoutData
from behavior_analysis.analysis.eye_convergence import calculate_convergence
import pandas as pd
from matplotlib import pyplot as plt
from scipy.spatial.distance import squareform
from pathlib import Path
import numpy as np
from behavior_analysis.analysis.bout_mapping import calculate_distance_matrix_templates, interpolate_nd
from matplotlib import pyplot as plt

In [ ]:
if __name__ == "__main__":

    experiment = BehaviorExperiment.open(r"C:\Users\manyung.ng\Documents\behaviour_analysis\behavior_analysis_tracking\test_analysis")
    print(experiment)
    bouts_path = experiment.subdirs['analysis'].joinpath('bouts.csv')
    bouts_df = pd.read_csv(bouts_path, dtype={'ID': str, 'video_code': str})
    video_info = pd.read_csv(experiment.directory.joinpath('video_data.csv'), dtype={'ID': str, 'video_code': str})

    # Average eye convergence over 20 ms
    window = 0.02

    # Import convergence data
    fish_convergence = pd.read_csv('', dtype={'ID': str})
    convergence_states = []
    # Import bout data
    bouts = BoutData.from_metadata(bouts_df, experiment.subdirs['kinematics'], tail_only=False)
    for i, bout in bouts.iter():
        # Bout info
        bout_info = bouts.metadata.loc[i]
        fps = video_info[video_info["code"] == bout_info.code].squeeze().fps
        ID = bout_info.ID
        fish_info = fish_convergence[fish_convergence["ID"] == ID].squeeze()
        # Calculate convergence
        bout_convergence = np.degrees(calculate_convergence(bout))
        w = int(window * fps)
        convergence_start = bout_convergence[:w].mean()
        convergence_end = bout_convergence[-w:].mean()
        convergence_states.append(np.array([convergence_start, convergence_end]) >= fish_info.threshold)
    convergence_states = np.array(convergence_states)
    # Find bout phases
    spontaneous = (~convergence_states[:, 0]) & (~convergence_states[:, 1])
    early = (~convergence_states[:, 0]) & (convergence_states[:, 1])
    mid = (convergence_states[:, 0]) & (convergence_states[:, 1])
    late = (convergence_states[:, 0]) & (~convergence_states[:, 1])
    phase_labels = np.column_stack([spontaneous, early, mid, late])
    phase_labels = np.argwhere(phase_labels)[:, 1]
    bouts_df['phase'] = phase_labels
    bouts_df.to_csv(experiment.subdirs["analysis"].joinpath("bout_convergence_labels.csv"), index=False)


## Behaviour analysis (Mearns et al., 2020)

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.spatial import distance as ssd
from matplotlib import pyplot as plt

In [ ]:
plt.rcParams["figure.dpi"] = 150

In [ ]:
# set data path
data_path = Path(r"C:\Users\manyung.ng\Documents\behaviour_analysis\behavior_analysis_tracking\test_analysis")

In [ ]:
# import metadata
md = pd.read_csv(data_path.joinpath("mapped_bouts.csv"), index_col="bout_index", dtype={"ID": str, "video_code": str})

In [ ]:
md

In [ ]:
bout_md = md.loc[0]
bout_md

In [ ]:
trial_path = data_path.joinpath("kinematics", bout_md["ID"], bout_md["video_code"] + ".csv")
trial_kinematics = pd.read_csv(trial_path)